## Import Dependencies

In [1]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv
from typing import List, Dict, Any
import time
from fastapi import FastAPI, HTTPException, Request, Depends
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import uvicorn

## Error handling for missing Dependencies

In [3]:
try:
    from langchain_community.vectorstores import FAISS
    from langchain_huggingface import HuggingFaceEmbeddings
    from langchain_community.document_loaders import TextLoader
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    from langchain.chains import RetrievalQA
    import google.generativeai as genai
    from langchain_google_genai import ChatGoogleGenerativeAI
    from fastapi import FastAPI, HTTPException, Request, Depends
    from fastapi.middleware.cors import CORSMiddleware
    import uvicorn
except ImportError as e:
    print(f"Error: Missing dependency - {e}")
    print("Please install required packages with: pip install langchain-google-genai google-generativeai langchain-huggingface faiss-cpu python-dotenv fastapi uvicorn")
    exit(1)

c:\Users\Dell 5480\Desktop\ERPNext chatbot\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Initialize fast API

In [4]:
app = FastAPI(
    title="ERP Document Chatbot API",
    description="API for querying ERP documents using Gemini and vector database",
    version="1.0.0"
)

## Enable CORS

In [5]:
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Allow all origins
    allow_credentials=True,
    allow_methods=["*"],  # Allow all methods
    allow_headers=["*"],  # Allow all headers
)

## Define Request & Response Model

In [6]:
class ChatbotRequest(BaseModel):
    query: str

# Define response model
class ChatbotResponse(BaseModel):
    answer: str
    sources: List[str] = []
    response_time: float

# Global variables for persistent components
vectordb = None
qa_chain = None

## Load Documents Function

In [7]:
def load_documents(docs_dir: str) -> List:
    """Load documents from the specified directory"""
    try:
        all_docs = []
        for root, _, files in os.walk(docs_dir):
            for file in files:
                if file.endswith(".txt"):
                    try:
                        loader = TextLoader(os.path.join(root, file))
                        all_docs.extend(loader.load())
                        print(f"Loaded: {os.path.join(root, file)}")
                    except Exception as e:
                        print(f"Error loading {file}: {str(e)}")
        
        if not all_docs:
            print(f"Warning: No documents found in {docs_dir}")
            
        return all_docs
    except Exception as e:
        print(f"Error loading documents: {str(e)}")
        raise

## Setup Vector Database

In [8]:
def setup_vectordb(documents: List, save_path: str = "vectorstore") -> Any:
    """Create or load vector database"""
    try:
        # Check if we already have a saved vector store
        if os.path.exists(save_path):
            print(f"Loading existing vector store from {save_path}")
            embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
            return FAISS.load_local(save_path, embedding_model)
        
        # Create new vector store
        print("Creating new vector store...")
        splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        split_docs = splitter.split_documents(documents)
        print(f"Split into {len(split_docs)} chunks")
        
        embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
        vectordb = FAISS.from_documents(split_docs, embedding_model)
        
        # Save for future use
        print(f"Saving vector store to {save_path}")
        vectordb.save_local(save_path)
        
        return vectordb
    except Exception as e:
        print(f"Error setting up vector database: {str(e)}")
        raise

## Setup Gemini LLM 

In [9]:
def setup_gemini_llm():
    """Set up Google's Gemini model"""
    try:
        print("Setting up Gemini model...")
        
        # Get API key from environment variables
        google_api_key = os.getenv("GOOGLE_API_KEY")
        if not google_api_key:
            raise ValueError("GOOGLE_API_KEY not found in environment variables")
        
        # Configure the Gemini model
        genai.configure(api_key=google_api_key)
        
        # define model
        model_name = "gemini-1.5-pro-latest"
        
        # Create LangChain wrapper for Gemini
        llm = ChatGoogleGenerativeAI(
            model=model_name,
            temperature=0.1,
        )
        
        return llm
    except Exception as e:
        print(f"Error setting up Gemini model: {str(e)}")
        raise

## Define QA Chain

In [10]:
def get_qa_chain():
    """Initialize or return the QA chain"""
    global qa_chain, vectordb
    
    if qa_chain is not None:
        return qa_chain
    
    # Check if vectordb is already initialized
    if vectordb is None:
        # Load documents 
        docs_dir = "erp_docs"
        if not os.path.exists(docs_dir):
            raise FileNotFoundError(f"Directory {docs_dir} not found")
        
        all_docs = load_documents(docs_dir)
        vectordb = setup_vectordb(all_docs)
    
    # Set up Gemini LLM
    llm = setup_gemini_llm()
    
    # Create retriever and QA chain
    retriever = vectordb.as_retriever(search_kwargs={"k": 3})
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm, 
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True
    )
    
    return qa_chain

## Startup Event

In [11]:
@app.on_event("startup")
async def startup_event():
    """Initialize components on startup"""
    try:
        print("Initializing chatbot components...")
        get_qa_chain()
        print("ERP Document Chatbot API is ready!")
    except Exception as e:
        print(f"Error during initialization: {str(e)}")
        # We'll continue without failing, and initialize when needed

C:\Users\Dell 5480\AppData\Local\Temp\ipykernel_18824\3668355532.py:1: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")


## Health Check Endpoint

In [12]:
@app.get("/health")
async def health_check():
    return {"status": "healthy", "message": "ERP Document Chatbot API is running"}


## Main endpoint for chatbot

In [13]:
@app.post("/chat", response_model=ChatbotResponse)
async def chat(request: ChatbotRequest):
    try:
        start_time = time.time()
        
        # Make sure QA chain is initialized
        qa = get_qa_chain()
        
        # Process the query
        response = qa.invoke({"query": request.query})
        answer = response.get("result", "No answer found")
        
        # Get sources
        sources = []
        if "source_documents" in response:
            for doc in response["source_documents"][:3]:  # Top 3 sources
                source = doc.metadata.get("source", "Unknown source")
                sources.append(os.path.basename(source))
        
        # Calculate response time
        response_time = time.time() - start_time
        
        return ChatbotResponse(
            answer=answer,
            sources=sources,
            response_time=response_time
        )
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error processing query: {str(e)}")


## Check for available Models

In [14]:
@app.get("/models")
async def list_models():
    try:
        # Get API key from environment variables
        google_api_key = os.getenv("GOOGLE_API_KEY")
        if not google_api_key:
            raise ValueError("GOOGLE_API_KEY not found in environment variables")
        
        # Configure the Gemini API
        genai.configure(api_key=google_api_key)
        
        # List available models
        models = []
        for model in genai.list_models():
            models.append({
                "name": model.name,
                "supported_methods": model.supported_generation_methods
            })
        
        return {"models": models}
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error listing models: {str(e)}")


## Main

In [ ]:
if __name__ == "__main__":
    uvicorn.run("fastAPI:app", host="0.0.0.0", port=8000, reload=True)

INFO:     Will watch for changes in these directories: ['c:\\Users\\Dell 5480\\Desktop\\ERPNext chatbot']
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [18824] using StatReload
